In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# read my favorite books
liked_books = pd.read_csv("liked_books.csv", index_col=0)

In [3]:
liked_books.head()

,user_id,book_id,rating,title
0,-1,69571,5,"Rich Dad, Poor Dad"
1,-1,2249,5,Seven Habits of Highly Effective People
2,-1,4894,4,Who Moved My Cheese?
3,-1,1145278,4,Highlighted in Yellow: A Short Course In Livin...
5,-1,54270,3,Mein Kampf


In [4]:
book_mapping = pd.read_csv('book_id_map.csv')

In [5]:
book_mapping.head()

,book_id_csv,book_id
0,0,34684622
1,1,34536488
2,2,34017076
3,3,71730
4,4,30422361


In [6]:
liked_books['book_id'] = liked_books['book_id'].astype(str)
liked_books['user_id'] = liked_books['user_id'].astype(str)

In [7]:
book_mapping['book_id'] = book_mapping['book_id'].astype(str)
book_mapping['book_id_csv'] = book_mapping['book_id_csv'].astype(str)

In [8]:
liked_books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 0 to 5
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  5 non-null      object
 1   book_id  5 non-null      object
 2   rating   5 non-null      int64 
 3   title    5 non-null      object
dtypes: int64(1), object(3)
memory usage: 200.0+ bytes


In [9]:
liked_books

,user_id,book_id,rating,title
0,-1,69571,5,"Rich Dad, Poor Dad"
1,-1,2249,5,Seven Habits of Highly Effective People
2,-1,4894,4,Who Moved My Cheese?
3,-1,1145278,4,Highlighted in Yellow: A Short Course In Livin...
5,-1,54270,3,Mein Kampf


In [10]:
book_mapping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2360650 entries, 0 to 2360649
Data columns (total 2 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   book_id_csv  object
 1   book_id      object
dtypes: object(2)
memory usage: 36.0+ MB


In [11]:
# convert the book_mapping dataframe to a dictionary
book_mapping = dict(zip(book_mapping['book_id_csv'], book_mapping['book_id']))

In [12]:
# show the length of the book_mapping dictionary
len(book_mapping)

2360650

In [13]:
book_set = set(liked_books["book_id"])

In [14]:
# find the similarity users who liked the same books
overlap_users = {}
with open('goodreads_interactions.csv', 'r') as file:
    for line in file:
        user_id, csv_id, _, rating, _ = line.split(',')
        

        book_id = book_mapping.get(csv_id)
        if book_id in book_set:
            if user_id not in overlap_users:
                overlap_users[user_id] = 1
            else:
                overlap_users[user_id] += 1

In [16]:
len(overlap_users)

36838

In [21]:
# add people who just read greater than 100 like us
filtered_overlap_users = set([k for k in overlap_users if overlap_users[k] > liked_books.shape[0]/50])

In [22]:
len(filtered_overlap_users)

36838

In [23]:
interactions_list = []
with open('goodreads_interactions.csv', 'r') as file:
    for line in file:
        user_id, csv_id, _, rating, _ = line.split(',')
        

        if user_id in filtered_overlap_users:
            book_id = book_mapping.get(csv_id)
            interactions_list.append([user_id, book_id, rating])

In [24]:
len(interactions_list)

18624784

In [25]:
interactions = pd.DataFrame(interactions_list, columns=["user_id", "book_id", "rating"])

In [26]:
interactions = pd.concat([liked_books[["user_id", "book_id", "rating"]], interactions])

In [27]:
interactions

,user_id,book_id,rating
0,-1,69571,5
1,-1,2249,5
2,-1,4894,4
3,-1,1145278,4
5,-1,54270,3
...,...,...,...
18624779,876126,5357,0
18624780,876126,1205297,0
18624781,876126,5347,4
18624782,876126,13154952,0


In [28]:
interactions['book_id'] = interactions['book_id'].astype(str)
interactions['user_id'] = interactions['user_id'].astype(str)

In [29]:
interactions['book_id'].unique()

array(['69571', '2249', '4894', ..., '980469', '17202865', '28455302'],
      dtype=object)

In [30]:
interactions['user_id'].unique()

array(['-1', '20', '56', ..., '876099', '876120', '876126'], dtype=object)

In [31]:
interactions['rating'] = pd.to_numeric(interactions['rating'])

In [32]:
interactions['book_index'] = interactions['book_id'].astype("category").cat.codes
interactions['user_index'] = interactions['user_id'].astype("category").cat.codes

In [33]:
interactions['book_index'].unique()

array([1142512,  567178, 1027204, ..., 1302327,  318038,  797146])

In [34]:
interactions['user_index'].unique()

array([    0,  6358, 25991, ..., 36030, 36031, 36032])

In [35]:
from scipy.sparse import coo_matrix
ratings_mat_coo = coo_matrix((interactions['rating'], (interactions['user_index'], interactions['book_index'])))

In [36]:
ratings_mat_coo

<36839x1311735 sparse matrix of type '<class 'numpy.int64'>'
	with 18624789 stored elements in COOrdinate format>

In [37]:
ratings_mat = ratings_mat_coo.tocsr()

In [38]:
interactions[interactions['user_id'] == '-1']

,user_id,book_id,rating,book_index,user_index
0,-1,69571,5,1142512,0
1,-1,2249,5,567178,0
2,-1,4894,4,1027204,0
3,-1,1145278,4,68649,0
5,-1,54270,3,1045444,0


In [39]:
my_index = 0

In [40]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(ratings_mat[my_index, :], ratings_mat).flatten()

In [41]:
similarity[0]

1.0000000000000002

In [42]:
import numpy as np
indices = np.argpartition(similarity, -50)[-50:]

In [43]:
indices

array([31155, 31853, 34401, 32750, 22555, 28609, 32075, 33764, 30273,
       26753, 29538, 33740, 29618, 33105, 23765, 11397, 34086, 31721,
       24512, 22827, 30501, 33712, 35610, 26816, 33572, 22070, 23063,
       31248, 34533, 33882, 24593, 25560, 25736, 25892, 33133, 17831,
       33670, 28203, 29119, 34682, 30365, 33775, 30938, 34303, 34293,
       34213, 33832, 31736, 32534,     0], dtype=int64)

In [44]:
similar_users = interactions[interactions['user_index'].isin(indices)].copy()

In [45]:
similar_users = similar_users[similar_users['user_index'] != '-1']

In [46]:
similar_users

,user_id,book_id,rating,book_index,user_index
0,-1,69571,5,1142512,0
1,-1,2249,5,567178,0
2,-1,4894,4,1027204,0
3,-1,1145278,4,68649,0
5,-1,54270,3,1045444,0
...,...,...,...,...,...
18560037,836544,23972780,0,641105,34533
18604316,864565,6346975,2,1091950,35610
18604317,864565,69571,4,1142512,35610
18604318,864565,4894,4,1027204,35610


In [65]:
book_rec = similar_users.groupby('book_id').rating.agg(["count", "mean"])

In [66]:
book_rec

,count,mean
book_id,,
1,2,0.000000
100223,1,0.000000
1005,6,0.833333
100915,1,0.000000
10127019,3,0.000000
...,...,...
994156,1,0.000000
99664,1,0.000000
9969571,1,0.000000


In [67]:
book_titles = pd.read_csv('D:\\Data_Science\\My projects in Data Science and ML\\extra files for book recommendation\\titles.csv')

In [68]:
book_titles

,book_id,title,rating,url,image_url,average_rating,edit_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,4.03,the unschooled wizard sun wolf and starhawk 12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,3.49,best friends forever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,4.13,the aeneid for boys and girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,4.22,alls fairy in love and war avalon web of magic 8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,3.81,the devils notebook
...,...,...,...,...,...,...,...
1308952,17805813,"Ondine (Ondine Quartet, #0.5)",327,https://www.goodreads.com/book/show/17805813-o...,https://images.gr-assets.com/books/1379766592m...,4.02,ondine ondine quartet 05
1308953,331839,Jacqueline Kennedy Onassis: Friend of the Arts,18,https://www.goodreads.com/book/show/331839.Jac...,https://s.gr-assets.com/assets/nophoto/book/11...,4.36,jacqueline kennedy onassis friend of the arts
1308954,2685097,The Spaniard's Blackmailed Bride,112,https://www.goodreads.com/book/show/2685097-th...,https://s.gr-assets.com/assets/nophoto/book/11...,3.42,the spaniards blackmailed bride
1308955,2342551,The Children's Classic Poetry Collection,36,https://www.goodreads.com/book/show/2342551.Th...,https://s.gr-assets.com/assets/nophoto/book/11...,4.00,the childrens classic poetry collection


In [69]:
book_titles['book_id'] = book_titles['book_id'].astype(str)

In [70]:
book_rec = book_rec.merge(book_titles, how='inner', on='book_id')

In [71]:
book_rec

,book_id,count,mean,title,rating,url,image_url,average_rating,edit_title
0,1,2,0.000000,Harry Potter and the Half-Blood Prince (Harry ...,1713866,https://www.goodreads.com/book/show/1.Harry_Po...,https://images.gr-assets.com/books/1361039191m...,4.54,harry potter and the halfblood prince harry po...
1,100223,1,0.000000,The Preservationist,968,https://www.goodreads.com/book/show/100223.The...,https://images.gr-assets.com/books/1316637488m...,3.64,the preservationist
2,1005,6,0.833333,Think and Grow Rich,87634,https://www.goodreads.com/book/show/1005.Think...,https://s.gr-assets.com/assets/nophoto/book/11...,4.17,think and grow rich
3,100915,1,0.000000,"The Lion, the Witch, and the Wardrobe (Chronic...",1575387,https://www.goodreads.com/book/show/100915.The...,https://images.gr-assets.com/books/1353029077m...,4.19,the lion the witch and the wardrobe chronicles...
4,10127019,3,0.000000,The Lean Startup: How Today's Entrepreneurs Us...,95622,https://www.goodreads.com/book/show/10127019-t...,https://images.gr-assets.com/books/1333576876m...,4.04,the lean startup how todays entrepreneurs use ...
...,...,...,...,...,...,...,...,...,...
852,994156,1,0.000000,"Supernova: Ksatria, Puteri, dan Bintang Jatuh",10473,https://www.goodreads.com/book/show/994156.Sup...,https://s.gr-assets.com/assets/nophoto/book/11...,3.83,supernova ksatria puteri dan bintang jatuh
853,99664,1,0.000000,The Painted Veil,24606,https://www.goodreads.com/book/show/99664.The_...,https://images.gr-assets.com/books/1320421719m...,3.91,the painted veil
854,9969571,1,0.000000,Ready Player One,376328,https://www.goodreads.com/book/show/9969571-re...,https://images.gr-assets.com/books/1500930947m...,4.31,ready player one
855,998,2,0.000000,The Millionaire Next Door: The Surprising Secr...,46032,https://www.goodreads.com/book/show/998.The_Mi...,https://s.gr-assets.com/assets/nophoto/book/11...,4.00,the millionaire next door the surprising secre...


In [72]:
book_rec['adjusted_count'] = book_rec['count'] * (book_rec['count'] / book_rec['rating'])

In [73]:
book_rec['score'] = book_rec['mean'] * book_rec['adjusted_count']

In [74]:
book_rec = book_rec[~ book_rec['book_id'].isin(liked_books['book_id'])]

In [75]:
liked_books['edit_title'] = (
    liked_books['title']
    .str.replace('[^a-zA-Z0-9 ]', '', regex=True)
    .str.lower()
    .str.strip()
)

In [76]:
liked_books

,user_id,book_id,rating,title,edit_title
0,-1,69571,5,"Rich Dad, Poor Dad",rich dad poor dad
1,-1,2249,5,Seven Habits of Highly Effective People,seven habits of highly effective people
2,-1,4894,4,Who Moved My Cheese?,who moved my cheese
3,-1,1145278,4,Highlighted in Yellow: A Short Course In Livin...,highlighted in yellow a short course in livin
5,-1,54270,3,Mein Kampf,mein kampf


In [77]:
book_rec = book_rec[~ book_rec['edit_title'].isin(liked_books['edit_title'])]

In [78]:
book_rec

,book_id,count,mean,title,rating,url,image_url,average_rating,edit_title,adjusted_count,score
0,1,2,0.000000,Harry Potter and the Half-Blood Prince (Harry ...,1713866,https://www.goodreads.com/book/show/1.Harry_Po...,https://images.gr-assets.com/books/1361039191m...,4.54,harry potter and the halfblood prince harry po...,2.333905e-06,0.000000
1,100223,1,0.000000,The Preservationist,968,https://www.goodreads.com/book/show/100223.The...,https://images.gr-assets.com/books/1316637488m...,3.64,the preservationist,1.033058e-03,0.000000
2,1005,6,0.833333,Think and Grow Rich,87634,https://www.goodreads.com/book/show/1005.Think...,https://s.gr-assets.com/assets/nophoto/book/11...,4.17,think and grow rich,4.107995e-04,0.000342
3,100915,1,0.000000,"The Lion, the Witch, and the Wardrobe (Chronic...",1575387,https://www.goodreads.com/book/show/100915.The...,https://images.gr-assets.com/books/1353029077m...,4.19,the lion the witch and the wardrobe chronicles...,6.347647e-07,0.000000
4,10127019,3,0.000000,The Lean Startup: How Today's Entrepreneurs Us...,95622,https://www.goodreads.com/book/show/10127019-t...,https://images.gr-assets.com/books/1333576876m...,4.04,the lean startup how todays entrepreneurs use ...,9.412060e-05,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
852,994156,1,0.000000,"Supernova: Ksatria, Puteri, dan Bintang Jatuh",10473,https://www.goodreads.com/book/show/994156.Sup...,https://s.gr-assets.com/assets/nophoto/book/11...,3.83,supernova ksatria puteri dan bintang jatuh,9.548362e-05,0.000000
853,99664,1,0.000000,The Painted Veil,24606,https://www.goodreads.com/book/show/99664.The_...,https://images.gr-assets.com/books/1320421719m...,3.91,the painted veil,4.064049e-05,0.000000
854,9969571,1,0.000000,Ready Player One,376328,https://www.goodreads.com/book/show/9969571-re...,https://images.gr-assets.com/books/1500930947m...,4.31,ready player one,2.657256e-06,0.000000
855,998,2,0.000000,The Millionaire Next Door: The Surprising Secr...,46032,https://www.goodreads.com/book/show/998.The_Mi...,https://s.gr-assets.com/assets/nophoto/book/11...,4.00,the millionaire next door the surprising secre...,8.689607e-05,0.000000


In [79]:
book_rec = book_rec[book_rec['count']>2]

In [80]:
book_rec

,book_id,count,mean,title,rating,url,image_url,average_rating,edit_title,adjusted_count,score
2,1005,6,0.833333,Think and Grow Rich,87634,https://www.goodreads.com/book/show/1005.Think...,https://s.gr-assets.com/assets/nophoto/book/11...,4.17,think and grow rich,0.000411,0.000342
4,10127019,3,0.000000,The Lean Startup: How Today's Entrepreneurs Us...,95622,https://www.goodreads.com/book/show/10127019-t...,https://images.gr-assets.com/books/1333576876m...,4.04,the lean startup how todays entrepreneurs use ...,0.000094,0.000000
19,10534,3,1.333333,The Art of War,174266,https://www.goodreads.com/book/show/10534.The_...,https://images.gr-assets.com/books/1453417993m...,3.96,the art of war,0.000052,0.000069
28,106835,3,0.000000,The Intelligent Investor,32476,https://www.goodreads.com/book/show/106835.The...,https://images.gr-assets.com/books/1409602421m...,4.26,the intelligent investor,0.000277,0.000000
45,11084145,5,0.600000,Steve Jobs,594067,https://www.goodreads.com/book/show/11084145-s...,https://images.gr-assets.com/books/1327861368m...,4.10,steve jobs,0.000042,0.000025
82,1202,6,0.000000,Freakonomics: A Rogue Economist Explores the H...,529274,https://www.goodreads.com/book/show/1202.Freak...,https://images.gr-assets.com/books/1327909092m...,3.94,freakonomics a rogue economist explores the hi...,0.000068,0.000000
96,12609433,7,0.714286,The Power of Habit: Why We Do What We Do in Li...,160604,https://www.goodreads.com/book/show/12609433-t...,https://images.gr-assets.com/books/1366758683m...,4.03,the power of habit why we do what we do in lif...,0.000305,0.000218
190,15843166,3,0.000000,You Are a Badass: How to Stop Doubting Your Gr...,34323,https://www.goodreads.com/book/show/15843166-y...,https://images.gr-assets.com/books/1351539051m...,4.03,you are a badass how to stop doubting your gre...,0.000262,0.000000
198,16071764,4,3.000000,"Lean In: Women, Work, and the Will to Lead",141600,https://www.goodreads.com/book/show/16071764-l...,https://images.gr-assets.com/books/1364250803m...,3.94,lean in women work and the will to lead,0.000113,0.000339
222,1713426,3,0.000000,Predictably Irrational: The Hidden Forces That...,63635,https://www.goodreads.com/book/show/1713426.Pr...,https://s.gr-assets.com/assets/nophoto/book/11...,4.10,predictably irrational the hidden forces that ...,0.000141,0.000000


In [84]:
# book_rec = book_rec[book_rec['mean']>4]

In [85]:
book_rec

,book_id,count,mean,title,rating,url,image_url,average_rating,edit_title,adjusted_count,score
2,1005,6,0.833333,Think and Grow Rich,87634,https://www.goodreads.com/book/show/1005.Think...,https://s.gr-assets.com/assets/nophoto/book/11...,4.17,think and grow rich,0.000411,0.000342
4,10127019,3,0.000000,The Lean Startup: How Today's Entrepreneurs Us...,95622,https://www.goodreads.com/book/show/10127019-t...,https://images.gr-assets.com/books/1333576876m...,4.04,the lean startup how todays entrepreneurs use ...,0.000094,0.000000
19,10534,3,1.333333,The Art of War,174266,https://www.goodreads.com/book/show/10534.The_...,https://images.gr-assets.com/books/1453417993m...,3.96,the art of war,0.000052,0.000069
28,106835,3,0.000000,The Intelligent Investor,32476,https://www.goodreads.com/book/show/106835.The...,https://images.gr-assets.com/books/1409602421m...,4.26,the intelligent investor,0.000277,0.000000
45,11084145,5,0.600000,Steve Jobs,594067,https://www.goodreads.com/book/show/11084145-s...,https://images.gr-assets.com/books/1327861368m...,4.10,steve jobs,0.000042,0.000025
82,1202,6,0.000000,Freakonomics: A Rogue Economist Explores the H...,529274,https://www.goodreads.com/book/show/1202.Freak...,https://images.gr-assets.com/books/1327909092m...,3.94,freakonomics a rogue economist explores the hi...,0.000068,0.000000
96,12609433,7,0.714286,The Power of Habit: Why We Do What We Do in Li...,160604,https://www.goodreads.com/book/show/12609433-t...,https://images.gr-assets.com/books/1366758683m...,4.03,the power of habit why we do what we do in lif...,0.000305,0.000218
190,15843166,3,0.000000,You Are a Badass: How to Stop Doubting Your Gr...,34323,https://www.goodreads.com/book/show/15843166-y...,https://images.gr-assets.com/books/1351539051m...,4.03,you are a badass how to stop doubting your gre...,0.000262,0.000000
198,16071764,4,3.000000,"Lean In: Women, Work, and the Will to Lead",141600,https://www.goodreads.com/book/show/16071764-l...,https://images.gr-assets.com/books/1364250803m...,3.94,lean in women work and the will to lead,0.000113,0.000339
222,1713426,3,0.000000,Predictably Irrational: The Hidden Forces That...,63635,https://www.goodreads.com/book/show/1713426.Pr...,https://s.gr-assets.com/assets/nophoto/book/11...,4.10,predictably irrational the hidden forces that ...,0.000141,0.000000


In [86]:
top_rec = book_rec.sort_values('score', ascending=False)

In [87]:
top_rec

,book_id,count,mean,title,rating,url,image_url,average_rating,edit_title,adjusted_count,score
523,356896,3,1.666667,The Greatest Salesman in the World,37319,https://www.goodreads.com/book/show/356896.The...,https://images.gr-assets.com/books/1403192719m...,4.19,the greatest salesman in the world,0.000241,0.000402
773,81922,3,1.000000,Rich Dad's Cashflow Quadrant: Rich Dad's Guide...,25540,https://www.goodreads.com/book/show/81922.Rich...,https://images.gr-assets.com/books/1483023081m...,4.06,rich dads cashflow quadrant rich dads guide to...,0.000352,0.000352
2,1005,6,0.833333,Think and Grow Rich,87634,https://www.goodreads.com/book/show/1005.Think...,https://s.gr-assets.com/assets/nophoto/book/11...,4.17,think and grow rich,0.000411,0.000342
198,16071764,4,3.000000,"Lean In: Women, Work, and the Will to Lead",141600,https://www.goodreads.com/book/show/16071764-l...,https://images.gr-assets.com/books/1364250803m...,3.94,lean in women work and the will to lead,0.000113,0.000339
96,12609433,7,0.714286,The Power of Habit: Why We Do What We Do in Li...,160604,https://www.goodreads.com/book/show/12609433-t...,https://images.gr-assets.com/books/1366758683m...,4.03,the power of habit why we do what we do in lif...,0.000305,0.000218
681,6346975,3,0.666667,Moonwalking with Einstein: The Art and Science...,49971,https://www.goodreads.com/book/show/6346975-mo...,https://s.gr-assets.com/assets/nophoto/book/11...,3.85,moonwalking with einstein the art and science ...,0.000180,0.000120
701,6708,3,1.333333,The Power of Now: A Guide to Spiritual Enlight...,109355,https://www.goodreads.com/book/show/6708.The_P...,https://s.gr-assets.com/assets/nophoto/book/11...,4.11,the power of now a guide to spiritual enlighte...,0.000082,0.000110
529,36072,9,0.444444,The 7 Habits of Highly Effective People: Power...,330726,https://www.goodreads.com/book/show/36072.The_...,https://images.gr-assets.com/books/1421842784m...,4.05,the 7 habits of highly effective people powerf...,0.000245,0.000109
793,865,9,1.333333,The Alchemist,1342863,https://www.goodreads.com/book/show/865.The_Al...,https://images.gr-assets.com/books/1483412266m...,3.82,the alchemist,0.000060,0.000080
19,10534,3,1.333333,The Art of War,174266,https://www.goodreads.com/book/show/10534.The_...,https://images.gr-assets.com/books/1453417993m...,3.96,the art of war,0.000052,0.000069


In [88]:
# Function to make the URLs clickable
def make_clickable(val):
    return '<a target="_blank" href="{}">click</a>'.format(val)

# Function to display images
def show_image(val):
    return '<img src="{}" width="100" height="100">'.format(val)

top_rec.style.format({'url': make_clickable, 'image_url': show_image})

,book_id,count,mean,title,rating,url,image_url,average_rating,edit_title,adjusted_count,score
523,356896,3,1.666667,The Greatest Salesman in the World,37319,click,,4.190000,the greatest salesman in the world,0.000241,0.000402
773,81922,3,1.000000,Rich Dad's Cashflow Quadrant: Rich Dad's Guide to Financial Freedom,25540,click,,4.060000,rich dads cashflow quadrant rich dads guide to financial freedom,0.000352,0.000352
2,1005,6,0.833333,Think and Grow Rich,87634,click,,4.170000,think and grow rich,0.000411,0.000342
198,16071764,4,3.000000,"Lean In: Women, Work, and the Will to Lead",141600,click,,3.940000,lean in women work and the will to lead,0.000113,0.000339
96,12609433,7,0.714286,The Power of Habit: Why We Do What We Do in Life and Business,160604,click,,4.030000,the power of habit why we do what we do in life and business,0.000305,0.000218
681,6346975,3,0.666667,Moonwalking with Einstein: The Art and Science of Remembering Everything,49971,click,,3.850000,moonwalking with einstein the art and science of remembering everything,0.000180,0.000120
701,6708,3,1.333333,The Power of Now: A Guide to Spiritual Enlightenment,109355,click,,4.110000,the power of now a guide to spiritual enlightenment,0.000082,0.000110
529,36072,9,0.444444,The 7 Habits of Highly Effective People: Powerful Lessons in Personal Change,330726,click,,4.050000,the 7 habits of highly effective people powerful lessons in personal change,0.000245,0.000109
793,865,9,1.333333,The Alchemist,1342863,click,,3.820000,the alchemist,0.000060,0.000080
19,10534,3,1.333333,The Art of War,174266,click,,3.960000,the art of war,0.000052,0.000069
